In [1]:
import torch

/home/wintoo/anaconda3/envs/gan/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
window_size=[7,7]
coords_h = torch.arange(window_size[0])
print("coords_h:\n",coords_h)
coords_w = torch.arange(window_size[1])
print("coords_w:\n",coords_w)

coords_h:
 tensor([0, 1, 2, 3, 4, 5, 6])
coords_w:
 tensor([0, 1, 2, 3, 4, 5, 6])


In [4]:
coords = torch.stack(torch.meshgrid([coords_h, coords_w]))  # 2, Wh, Ww
print("coords:\n",coords)
coords_flatten = torch.flatten(coords, 1)  # 2, Wh*Ww
print("coords_flatten:\n",coords_flatten)

coords:
 tensor([[[0, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1],
         [2, 2, 2, 2, 2, 2, 2],
         [3, 3, 3, 3, 3, 3, 3],
         [4, 4, 4, 4, 4, 4, 4],
         [5, 5, 5, 5, 5, 5, 5],
         [6, 6, 6, 6, 6, 6, 6]],

        [[0, 1, 2, 3, 4, 5, 6],
         [0, 1, 2, 3, 4, 5, 6],
         [0, 1, 2, 3, 4, 5, 6],
         [0, 1, 2, 3, 4, 5, 6],
         [0, 1, 2, 3, 4, 5, 6],
         [0, 1, 2, 3, 4, 5, 6],
         [0, 1, 2, 3, 4, 5, 6]]])
coords_flatten:
 tensor([[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3,
         3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6,
         6],
        [0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2,
         3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5,
         6]])


/home/wintoo/anaconda3/envs/gan/lib/python3.8/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755903507/work/aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [5]:
relative_coords = coords_flatten[:, :, None] - coords_flatten[:, None, :]  # 2, Wh*Ww, Wh*Ww
print("relative_coords_1:\n",relative_coords)
relative_coords = relative_coords.permute(1, 2, 0).contiguous()  # Wh*Ww, Wh*Ww, 2
print("relative_coords_2:\n",relative_coords)
relative_coords[:, :, 0] += window_size[0] - 1  # shift to start from 0
print("relative_coords[:, :, 0]:\n",relative_coords[:, :, 0])

relative_coords_1:
 tensor([[[ 0,  0,  0,  ..., -6, -6, -6],
         [ 0,  0,  0,  ..., -6, -6, -6],
         [ 0,  0,  0,  ..., -6, -6, -6],
         ...,
         [ 6,  6,  6,  ...,  0,  0,  0],
         [ 6,  6,  6,  ...,  0,  0,  0],
         [ 6,  6,  6,  ...,  0,  0,  0]],

        [[ 0, -1, -2,  ..., -4, -5, -6],
         [ 1,  0, -1,  ..., -3, -4, -5],
         [ 2,  1,  0,  ..., -2, -3, -4],
         ...,
         [ 4,  3,  2,  ...,  0, -1, -2],
         [ 5,  4,  3,  ...,  1,  0, -1],
         [ 6,  5,  4,  ...,  2,  1,  0]]])
relative_coords_2:
 tensor([[[ 0,  0],
         [ 0, -1],
         [ 0, -2],
         ...,
         [-6, -4],
         [-6, -5],
         [-6, -6]],

        [[ 0,  1],
         [ 0,  0],
         [ 0, -1],
         ...,
         [-6, -3],
         [-6, -4],
         [-6, -5]],

        [[ 0,  2],
         [ 0,  1],
         [ 0,  0],
         ...,
         [-6, -2],
         [-6, -3],
         [-6, -4]],

        ...,

        [[ 6,  4],
         [ 6,

In [6]:

relative_coords[:, :, 1] += window_size[1] - 1
print("relative_coords[:, :, 1]:\n",relative_coords[:, :, 1])

relative_coords[:, :, 1]:
 tensor([[ 6,  5,  4,  ...,  2,  1,  0],
        [ 7,  6,  5,  ...,  3,  2,  1],
        [ 8,  7,  6,  ...,  4,  3,  2],
        ...,
        [10,  9,  8,  ...,  6,  5,  4],
        [11, 10,  9,  ...,  7,  6,  5],
        [12, 11, 10,  ...,  8,  7,  6]])


In [7]:
relative_coords[:, :, 0] *= 2 * window_size[1] - 1
print("relative_coords[:, :, 0]:\n",relative_coords[:, :, 0])

relative_coords[:, :, 0]:
 tensor([[ 78,  78,  78,  ...,   0,   0,   0],
        [ 78,  78,  78,  ...,   0,   0,   0],
        [ 78,  78,  78,  ...,   0,   0,   0],
        ...,
        [156, 156, 156,  ...,  78,  78,  78],
        [156, 156, 156,  ...,  78,  78,  78],
        [156, 156, 156,  ...,  78,  78,  78]])


In [11]:
relative_position_index = relative_coords.sum(-1)  # Wh*Ww, Wh*Ww
print("relative_pos_index:\n",relative_position_index[0])
print("relative_pos_index:\n",relative_position_index[1])
print("relative_pos_index:\n",relative_position_index[2])
print("relative_pos_index:\n",relative_position_index[:, 0].T)

relative_pos_index:
 tensor([84, 83, 82, 81, 80, 79, 78, 71, 70, 69, 68, 67, 66, 65, 58, 57, 56, 55,
        54, 53, 52, 45, 44, 43, 42, 41, 40, 39, 32, 31, 30, 29, 28, 27, 26, 19,
        18, 17, 16, 15, 14, 13,  6,  5,  4,  3,  2,  1,  0])
relative_pos_index:
 tensor([85, 84, 83, 82, 81, 80, 79, 72, 71, 70, 69, 68, 67, 66, 59, 58, 57, 56,
        55, 54, 53, 46, 45, 44, 43, 42, 41, 40, 33, 32, 31, 30, 29, 28, 27, 20,
        19, 18, 17, 16, 15, 14,  7,  6,  5,  4,  3,  2,  1])
relative_pos_index:
 tensor([86, 85, 84, 83, 82, 81, 80, 73, 72, 71, 70, 69, 68, 67, 60, 59, 58, 57,
        56, 55, 54, 47, 46, 45, 44, 43, 42, 41, 34, 33, 32, 31, 30, 29, 28, 21,
        20, 19, 18, 17, 16, 15,  8,  7,  6,  5,  4,  3,  2])
relative_pos_index:
 tensor([ 84,  85,  86,  87,  88,  89,  90,  97,  98,  99, 100, 101, 102, 103,
        110, 111, 112, 113, 114, 115, 116, 123, 124, 125, 126, 127, 128, 129,
        136, 137, 138, 139, 140, 141, 142, 149, 150, 151, 152, 153, 154, 155,
        162, 163, 1

/tmp/ipykernel_81701/3618870177.py:5: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matricesor `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755903507/work/aten/src/ATen/native/TensorShape.cpp:2318.)
  print("relative_pos_index:\n",relative_position_index[:, 0].T)
